In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import mysql.connector
import datetime


# Conectarse a la base de datos
mydb = mysql.connector.connect(
    host="195.235.211.197",
    user="pc2_grupo5",
    password="Proyectos.23",
    database="pc2_grupo5"
)

# Crear un cursor para ejecutar las consultas SQL
cursor = mydb.cursor()



ejercicios = []
pasosdelejercicios = []
nombredelejercicio = []

    
msc = {}

#body = ["chest"]
body = ["chest","shoulders","triceps","traps","biceps","forearms","obliques","abdominals","quadriceps","calves","lats","lowerback","glutes","hamstrings","traps_middle"]

timestamp = datetime.datetime.now()


for mus in body:
    musculo_verificar = mus
    print(musculo_verificar)
    
    cursor.execute("SELECT id FROM muscles WHERE name = %s", (musculo_verificar,))
    result = cursor.fetchone()
    
    # Si el músculo no existe, insertarlo en la tabla "muscle"
    if not result:
        cursor.execute("INSERT INTO muscles (name, created_at) VALUES (%s, %s)", (musculo_verificar,timestamp))
        muscle_id = cursor.lastrowid
    else:
        muscle_id = result[0]  
    
    # URL de la página web a scrappear
    url = 'https://www.hevyapp.com/muscle/' + str(mus)

    # Realizar la petición HTTP y obtener el contenido de la página web
    response = requests.get(url)
    html = response.content

    # Crear el objeto BeautifulSoup y analizar el contenido HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Buscar los elementos que contienen los ejercicios por músculo
    msc[mus] = []
    
    muscles = soup.find_all('article', class_='exercises')
    
    links=[]
    

    # Recorrer cada músculo encontrado e imprimir los ejercicios correspondientes
    for muscle in muscles:
        # Buscar los enlaces de los ejercicios correspondientes a este músculo
        exercise = muscle.find('h4')
        videos = []
        steps = []
        
        exercise_page = exercise.find('a')
        link = exercise_page['href']
        links.append(link)
        response2 = requests.get(link)
        html2 = response2.content
        exercise_page = BeautifulSoup(html2, 'html.parser')
        video = exercise_page.find_all('video')
        for v in video:
            link_video = v['src']
            videos.append(link_video)

        descripciones = exercise_page.find("ol").find_all('li')
        muscle_steps = []  # Lista para almacenar los pasos del ejercicio actual
        for d in descripciones:
            muscle_steps.append(d.text)
        steps.append(muscle_steps)  # Agregar los pasos del ejercicio actual a la lista de pasos por músculo

        msc[mus].append({'name': exercise.text.replace('\n', '').replace('Copied', '').strip(), 'steps': steps, 'video': videos})   
    

        for ex in exercise:

            ej = ex.text.replace('\n', '').replace('Copied', '').strip()
             # Buscar si el ejercicio ya existe en la tabla "exercise" para el músculo actual
            cursor.execute("SELECT id FROM exercises WHERE name = %s AND muscle_id = %s", (ej, muscle_id))
            result = cursor.fetchone()
            

            
             # Si el ejercicio no existe, insertarlo en la tabla "exercise"
            if not result:
                if videos:
                    cursor.execute("INSERT INTO exercises (name, difficulty, url, muscle_id, created_at) VALUES (%s, %s, %s, %s, %s)", 
                           (ej, " ", videos[0], muscle_id, timestamp))
                else:
                    cursor.execute("INSERT INTO exercises (name, difficulty, url, muscle_id, created_at) VALUES (%s, %s, %s, %s, %s)", 
                           (ej, " ", "None", muscle_id, timestamp))

            else:
                exercise_id = result[0]
            
    
print(json.dumps(msc, indent=4))
with open('ejercicios.json', 'w') as f:
    json.dump(msc, f, indent=4)
    
    
# Hacer commit de las transacciones y cerrar la conexión
mydb.commit()
cursor.close()
mydb.close()

chest
shoulders
triceps
traps
biceps
forearms
obliques
abdominals
quadriceps
calves


In [ ]:
            response2 = requests.get('https://www.hevyapp.com/exercises/bench-press-dumbbell/')
            html2 = response2.content
            exercise_page = BeautifulSoup(html2, 'html.parser')
            video = exercise_page.find_all('video')
            videos = []
            for v in video:
                link_video = v['src']
                videos.append(link_video)
            descript = exercise_page.find_all('ol')
            for desc in descript:
                descripciones = desc.find_all('li')
                muscle_steps = []  # Lista para almacenar los pasos del ejercicio actual
                for d in descripciones:
                    muscle_steps.append(d.text)
                steps.append(muscle_steps)  # Agregar los pasos del ejercicio actual a la lista de pasos por músculo

In [ ]:
 musculo_verificar = mus
    print(musculo_verificar)
    
    cursor.execute("SELECT id FROM muscles WHERE name = %s", (musculo_verificar,))
    result = cursor.fetchone()
    
    # Si el músculo no existe, insertarlo en la tabla "muscle"
    if not result:
        cursor.execute("INSERT INTO muscles (name, created_at) VALUES (%s, %s)", (musculo_verificar,timestamp))
        muscle_id = cursor.lastrowid
    else:
        muscle_id = result[0]  